In [1]:
import pandas as pd
import os

In [2]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
"""Packagewise and Structurewise Summation"""
def packageWiseSum(package_df,projection_df):
    plist=list(package_df['Package_No'])
    for index,row in projection_df.iterrows():
        pname=row['Package']
        row_index=plist.index(pname)
        package_df.iloc[row_index,1]=package_df.iloc[row_index,1]+row['VDW_30_06_2020']
        package_df.iloc[row_index,2]=package_df.iloc[row_index,2]+row['VDW_30_06_2021']
    for index,row in package_df.iterrows():
        package_df.iloc[index,4]= package_df.iloc[index,2]- package_df.iloc[index,3]
    return package_df
def structureWiseSum(structure_df,projection_df):
    plist=list(structure_df['Code'])
    for index,row in projection_df.iterrows():
        pname=row['Structure Code']
        row_index=plist.index(pname)
        structure_df.iloc[row_index,3]=structure_df.iloc[row_index,3]+row['VDW_30_06_2020']
        structure_df.iloc[row_index,4]=structure_df.iloc[row_index,4]+row['VDW_30_06_2021']
    for index,row in structure_df.iterrows():
        structure_df.iloc[index,6]= structure_df.iloc[index,4]- structure_df.iloc[index,5]
    return structure_df

def dppWiseSum(structure_df,dpp_df):    
    dpp_item=list(dpp_df['DPP_Item']) #DPP_Item
    for index,row in structure_df.iterrows():
        #item_name=row[]
        pname=row['DPP_Item']
        row_index= dpp_item.index(pname)
        dpp_df.iloc[row_index,1]= dpp_df.iloc[row_index,1]+row['VWD_30_06_2020']
        dpp_df.iloc[row_index,2]=dpp_df.iloc[row_index,2]+row['VWD_30_06_2021']
    for index,row in dpp_df.iterrows():
        dpp_df.iloc[index,4]= dpp_df.iloc[index,2]- dpp_df.iloc[index,3]
       
    return dpp_df



In [4]:
def buildPackageWisePaymentMatrix(package_df):
    #Package_No Payment_30_06_2020
    package=list(package_df['Package_No'])
    paid=list(package_df['Payment_30_06_2020'])
    zero_list=[0 for x in package]
    data={'Package_No':package,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020':paid,'Payment_30_06_2021':zero_list}
    package_payment_df=pd.DataFrame(data)
    return package_payment_df
def buildStructureWisePaymentMatrix(structure_df):
    #DPP Item	Initial	Code
    structure_name=list(structure_df['Items'])
    structure_code=list(structure_df['Code'])
    DPP_Item_code=list(structure_df['DPP_Item'])  
    payment=list(structure_df['Payment_30_06_2020'])
    #Package_No Payment_30_06_2020    
    zero_list=[0 for x in structure_name]
    data={'Items':structure_name,'Code':structure_code ,'DPP_Item':DPP_Item_code,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020': payment,'Payment_30_06_2021':zero_list}
    structure_payment_df=pd.DataFrame(data)
    return structure_payment_df
    
def buildVWDMatrix(projection_df):
    #Component	Unit	Qnty	Cost	Present_Progress	Projected_progress
    #Package
    #Structure Code
    stc=list(projection_df['Structure Code'])
    packages=list(projection_df['Package'])
    components=list(projection_df['Component'])
    units=list(projection_df['Unit'])
    quantity=list(projection_df['Qnty'])
    cost=list(projection_df['Cost'])
    present_prog=list(projection_df['Present_Progress'])
    projected_prog=list(projection_df['Projected_progress'])
    zero_list=[0 for x in components ]
    data={'Package':packages,'Structure Code':stc,'Component':components,'Unit':units,'Qnty':quantity,'Cost':cost,
          'Present_Progress':present_prog,'Projected_progress':projected_prog,
          'VDW_30_06_2020': zero_list,'VDW_30_06_2021':zero_list,'Work_Done_20_21':zero_list,'Payment_30_06_2020': zero_list,
          'Payment_30_06_2020':zero_list}
    vwd_mat=pd.DataFrame(data)
    for index,row in vwd_mat.iterrows():
        vwd_mat.iloc[index,8]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,6]
        vwd_mat.iloc[index,9]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,7]
        vwd_mat.iloc[index,10]=vwd_mat.iloc[index,9]-vwd_mat.iloc[index,8]
    return vwd_mat
    


In [5]:
myfolder=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\RADP Preparations' #Home Computer
#myfolder=r'F:\website\cmis6\Civilworks cost\RADP_20_21'
input_file_name=os.path.join(myfolder,'Projecttion_input.xlsx')
out_file_name=os.path.join(myfolder,'Projecttion_output.xlsx')
myframes=[]
mynames=[]
sheetName="Structure_Type"
structure_type_df=pd.read_excel(input_file_name,sheet_name=sheetName)
structure_type_df.fillna(0,inplace=True)
structure_type_df
myframes.append(structure_type_df)
mynames.append("Structure_Type")
"""Projection Info"""
sheetName="Kishoregnj"
projection_info_df=pd.read_excel(input_file_name,sheet_name=sheetName)
projection_info_df.fillna(0,inplace=True)
projection_info_df
myframes.append(projection_info_df)
mynames.append("Projection_Info")
sheetName="Packages"
package_name_df=pd.read_excel(input_file_name,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
myframes.append(package_name_df)
mynames.append("Package_Name")
vwd_df=buildVWDMatrix(projection_info_df)
myframes.append(vwd_df)
mynames.append("Value_of_Work_Done")
package_payment_df=buildPackageWisePaymentMatrix(package_name_df)
package_payment_df=packageWiseSum(package_payment_df,vwd_df)
myframes.append(package_payment_df)
mynames.append("Package_wise_payment")
"""Building Structurewise Cost"""
structure_payment_df=buildStructureWisePaymentMatrix(structure_type_df)
structure_payment_df=structureWiseSum(structure_payment_df,vwd_df)
myframes.append(structure_payment_df)
mynames.append("Structure_Wise_payment")
"""Building Structurewise Cost"""
sheetName="DPP_Item"
dpp_item_df=pd.read_excel(input_file_name,sheet_name=sheetName)
dpp_item_df.fillna(0,inplace=True)
dpp_item_df=dppWiseSum(structure_payment_df,dpp_item_df)
myframes.append(dpp_item_df)
mynames.append("DPP_Wise_payment")
saveDataFrame(myframes,out_file_name,mynames)



In [6]:
projection_info_df

,Package Name,Component,Unit,Qnty,Cost,Present_Progress,Projected_progress,Package,Haor Type,Structure Code,Earth Volume,Hoar Code,District Code
0,BWDB/Kish/HFMLIP/ PW-01,Betal Doba from KM 1.315 to KM 3.466 =2.151 KM,Km,2.151,40.94,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-2,KISH
1,BWDB/Kish/HFMLIP/ PW-01,Alalia-bahadia Khal from KM 1.325 to KM 2.545=...,Km,1.220,35.88,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-1,KISH
2,BWDB/Kish/HFMLIP/ PW-01,katakhali khal from KM 0.000 to KM 1.550=1.550 KM,Km,1.550,19.18,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
3,BWDB/Kish/HFMLIP/ PW-01,Ganakkhali Khal from KM 0.000 to KM 5.625=5.62...,Km,5.625,309.80,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
4,BWDB/Kish/HFMLIP/ PW-01,kali Khal from KM 0.00 to KM 1.000=1.000 KM,Km,1.000,45.95,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,BWDB/Kish/HFMLIP/ PW-27,phuleshwari,Nos,1.000,156.62,0.0,1.0,KISH/PW-27,New,3,0.0,N-9,KISH
142,BWDB/Kish/HFMLIP/ PW-27,Kalna,Nos,1.000,148.00,0.0,1.0,KISH/PW-27,New,3,0.0,N-9,KISH
143,BWDB/Kish/HFMLIP/ PW-27,RCC Box Drainage,Nos,1.000,41.88,0.0,1.0,KISH/PW-27,New,4,0.0,N-9,KISH
144,BWDB/Kish/HFMLIP/ PW-27,Causeway,Nos,1.000,140.87,0.0,1.0,KISH/PW-27,New,5,0.0,N-9,KISH


In [7]:
dpp_item_df

,DPP_Item,VWD_30_06_2020,VWD_30_06_2021,Payment_30_06_2020,Payment_30_06_2021
0,Inlet,629.833788,646.433215,291.81,354.623215
1,Reg_Rehab_Rehab,0.000000,44.790000,0.00,44.790000
2,Reg_CW_Box,6776.296140,8964.130477,6503.97,2460.160477
3,Khal_Riv_New,2450.151093,3757.093345,2546.54,1210.553345
4,Khal_Riv_Rehab,76.820000,451.750000,51.09,400.660000
5,Full_Emb_Rehab,0.000000,0.000000,0.00,0.000000
6,Sub_Emb_Rehab,0.000000,0.000000,0.00,0.000000
7,Sub_Emb_Const,6093.142193,7331.220028,5833.04,1498.180028
8,Reg_Rehab_New,0.000000,120.556364,0.00,120.556364
9,Thr_Fl_Cost,0.000000,0.000000,0.00,0.000000
